In [45]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Yelp API Key
gkey = 'fxuoEjmZHd5kA1x20u4z2waSsXfTm49pWr2F6ac-htERJ0haCqXH949tlmDYt9lhy4MC6nBJjt2llMwhQJfxbwX69QlFkI221WkMDZTn7NMQBaXRrWGBFzCrg4-9XXYx'


In [46]:
from etl_clean_ip import zip_list

In [47]:
#set up base url
# base url
base_url = "https://api.yelp.com/v3/businesses/search"

# set up api_key dictionary
headers = {
        'Authorization': 'Bearer %s' % gkey,
    }

# set up a parameters dictionary
def s_params(target_loc,target_key,target_type):
    params = {
    "location": target_loc,
    "term": target_key,
    "categories": target_type,
}
    return params

# run a request using our params dictionary
def s_query(base_url,headers,params):
    response = requests.request('GET', base_url, headers=headers, params=params).json()
    return response

In [48]:
# Import zip list
zip_list 

#cities_pd.shape[0] #use .shape[0] to count the number of rows

[33142,
 33127,
 33147,
 33125,
 33136,
 33012,
 33167,
 33126,
 33150,
 33032,
 33130,
 33162,
 33034,
 33055,
 33056,
 33174,
 33172,
 33033,
 33168,
 33190,
 33169,
 33181,
 33030,
 33014,
 33193,
 33016,
 33161,
 33184,
 33194,
 33035]

In [49]:
# yelp function
def get_restaurants(zips):
    
    #these are the additional params
    add_params = { "limit" : 5, "radius" : 4000}

    #these are the original params
    loc = zips
    params = s_params(loc,"restaurants","restaurants, All")

    #add additional params to our params from before
    s1_params = {**params,**add_params}

    s1_results = s_query(base_url,headers,s1_params)
    
    if (s1_results["businesses"][0]["name"]):
        name = s1_results["businesses"][0]["name"]
        address = s1_results["businesses"][0]["location"]["display_address"]
        lat = s1_results["businesses"][0]["coordinates"]["latitude"]
        lon = s1_results["businesses"][0]["coordinates"]["longitude"]
        rating = s1_results["businesses"][0]["rating"]
        output = [name,address,lat,lon,rating] 
    else:
        output = ["NaN","NaN","NaN","NaN","NaN"]
    
    
    return output

In [50]:
# Loop through the zips list and run a search for each zip
names = []
addresses = []
lats = []
lons = []
ratings = []


print("wait...")


for zips in zip_list:


    output = get_restaurants(zips)
    print(output)
    names.append(output[0])
    addresses.append(output[1])
    lats.append(output[2])
    lons.append(output[3])
    ratings.append(output[4])
    
    
print("done!")

wait...
['Hometown Bar-B-Que', ['1200 NW 22nd St', 'Ste 100', 'Miami, FL 33142'], 25.797445333, -80.215986833, 4.5]
['KYU', ['251 NW 25th St', 'Miami, FL 33127'], 25.80089, -80.20017, 4.5]
['House of Food Porn', ['197 NW 62nd St', 'Miami, FL 33150'], 25.8326319884545, -80.200107379434, 5.0]
["Old's Havana Cuban Bar & Cocina", ['1442 SW 8th St', 'Miami, FL 33135'], 25.7655942148975, -80.2192383373502, 4.5]
["Old's Havana Cuban Bar & Cocina", ['1442 SW 8th St', 'Miami, FL 33135'], 25.7655942148975, -80.2192383373502, 4.5]
['Pan.com Sandwicheria', ['1255 W 46th St', 'Unit 18', 'Hialeah, FL 33012'], 25.86424, -80.30763, 4.5]
['El Conuquito Restaurant', ['1099 NW 119th St', 'Miami, FL 33168'], 25.8838214874268, -80.2164688110352, 4.5]
['First Watch', ['5799 NW 7th St', 'Miami, FL 33125'], 25.7783052848884, -80.2905444951525, 4.0]
['Sins Gastrobar', ['9835 NE 2nd Ave', 'Miami Shores, FL 33138'], 25.86562, -80.19331, 4.0]
['Figat Chefs Kitchen', ['22400 Old Dixie Hwy', 'Miami, FL 33170'], 25.

IndexError: list index out of range

In [ ]:
# Add columns for lat, lng, airport name, airport address, airport rating
cities_pd["Lat"] = lats
cities_pd["Lng"] = lons
cities_pd["Airport Name"] = names
cities_pd["Airport Address"] = addresses
cities_pd["Airport Rating"] = ratings
cities_pd.head()

In [ ]:
# Save Data to csv
cities_pd.to_csv("Airport_Output.csv")

# Visualize to confirm airport data appears
cities_pd.head(10)